In [ ]:
# default_exp sensors

# Sensors
> reads sensor data over UART


In [10]:
#export
import time
import serial
import numpy as np
import pandas as pd

In [11]:
#export

class sensor_stream(object):
    
    def __init__(self, baudrate=115200, port="/dev/ttyTHS0"):
        self.ser = serial.Serial(port=port,
                                baudrate=baudrate,
                                bytesize=serial.EIGHTBITS,
                                parity=serial.PARITY_NONE,
                                stopbits=serial.STOPBITS_ONE,
                                )
        
        # Initialise serial port and wait
        self.ser.flushInput()
        time.sleep(1) 
        
        # Instantiate list for storing data
        self.data = []
        self.data_df = None
        
        
    def request(self):
        
        # Signal teensy to return new data packet
        self.ser.write("\n".encode())
                       
                       
    def record(self):
        
        # Check if line is ready
        if self.ser.inWaiting() > 0:
            
            # Read line from serial
            line_data = self.ser.readline()
            
            # Format data
            line_data = str(line_data).replace("b", "").replace("'", "").split(",")[:-1]
            
            # Append line to list
            self.data.append(line_data)
            
    
    def to_df(self, fields_dict):
                
        # Convert to dataframe 
        self.data_df = pd.DataFrame(self.data, columns = fields_dict.keys())
        
        # Iterate through each column and update data type
        for field_name in fields_dict.keys():
            
            # Convert to float
            if fields_dict[field_name] == 'float' or fields_dict[field_name] == 'int':
                self.data_df[field_name] =  pd.to_numeric(self.data_df[field_name], errors='coerce')
            
            # Convert to datetime
            elif fields_dict[field_name] == 'datetime':
                self.data_df[field_name] =  pd.to_datetime(self.data_df[field_name], errors='coerce')
                
        # Return formatted data as dataframe
        return self.data_df

In [13]:
peripherals = sensor_stream(baudrate=921_600,port="/dev/ttyTHS0")
peripherals.request()
peripherals.record()

In [14]:
# Define dictionary column name and data type
data_dict = {'rtc_now': 'datetime', 
             'rtc_temp': 'float', 
             'air_temp': 'float', 
             'air_pressure': 'float', 
             'air_humidity': 'float', 
             'imu_cal': 'int',
             'imu_temp': 'float', 
             'euler_x': 'float', 
             'euler_y': 'float', 
             'euler_z': 'float', 
             'mag_x': 'float', 
             'mag_y': 'float', 
             'mag_z': 'float', 
             'gps_now': 'datetime', 
             'latitude': 'float', 
             'longitude': 'float', 
             'altitude': 'float', 
             'numSV': 'int', 
             'velN': 'float', 
             'velE': 'float', 
             'gSpeed': 'float', 
             'heading': 'float', 
             'velAcc': 'float', 
             'pDOP': 'float', 
             'hAcc': 'float', 
             'vAcc': 'float', 
             'headAcc': 'float', 
             'magDec': 'float', 
             'magAcc': 'float'}

#
data_df = peripherals.to_df(data_dict)